In [1]:
import os, json, numpy as np, matplotlib.pyplot as plt, torch
import lightkurve as lk
from models.cnn_autoencoder import CNNAutoencoder
from models.infer_ae import load_model, anomaly_score_series

DATA_PRO = "data/processed"
files = [f for f in os.listdir(DATA_PRO) if f.endswith("_flattened.csv")]
assert files, "Need flattened CSVs first."
path = os.path.join(DATA_PRO, files[0])
lc = lk.LightCurve.read(path)
flux = lc.flux.value.astype(np.float32)


ModuleNotFoundError: No module named 'models'